importo i pacchetti necessari

In [1]:
import pandas as pd
import requests
import coordinates_calcolator as cc
import url_contructor as uc
import image_visualizator as iv
from db_manager import create_empty_csv, populate_csv
import toml

           id     lat     lon   diam  depth
2   01-000002  69.244 -27.240  79.63   0.09
8   01-000008  75.539  13.829  58.40   0.11
10  01-000010  77.707 -27.314  52.37   0.07
15  01-000015  84.687   1.589  48.84   1.06
23  01-000023  84.452 -18.503  43.03   0.49
(85148, 5)


prendo i dati dal csv filtrato

In [2]:
mars_data = pd.read_csv('Mars_crater_db_filtered.csv')
zoom = 6
#create_empty_csv('Mars_crater_db_images.csv', ['id', 'image'])
toml_data = toml.load('secrets.toml')
api_key = toml_data['api_key']

dato un valore di inizio e uno di fine da mars data faccio partire una request costruita con url_contructor con lat e lon da coordinates_calcolator

In [3]:
def get_data(start, end, api_key):
    data = []
    for i in range(start, end):
        lat = mars_data['lat'][i]
        #print(lat)
        lon = mars_data['lon'][i]
        #print(lon)
        id = mars_data['id'][i]
        #print(id)
        lat, lon = cc.lat_lon_to_tile(lat, lon, zoom)
        #print (lat, lon)
        url = uc.get_request_url(lat, lon, tileMatrix=zoom)
        print(url)
        params = {'api_key': api_key}  # Aggiungi la chiave API come parametro
        
        # Esegui la richiesta HTTP con la chiave API
        response = requests.get(url, params=params)
       
        if response.status_code == 200:
            data.append([id,response.content])
        else:
            print('error')
    return data

In [7]:
data = get_data(0,20, api_key)


https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/27/14.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/34/10.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/27/9.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/32/0.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/28/1.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/30/14.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/33/14.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/33/15.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/32/10.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/35/14.png
https://trek.nasa.gov/tiles/Mars/


for i in range(len(data)):
    id = data[i][0]
    print(id)
    image = data[i][1]
    iv.image_visualizator_from_bin(image)


popolo il csv 'Mars_crater_db_images.csv' con i dati ottenuti

In [8]:
populate_csv(data)


Added 11 new entries. Total entries: 20 (before: 9)


In [6]:
print(len(data))

5
